<a href="https://colab.research.google.com/github/Fer071989/loopstructural_intrusions_tassiedolerites/blob/main/Model1_Feb2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/Fer071989/loopstructural_intrusions_tassiedolerites.git
    !pip install git+https://github.com/Loop3D/LoopStructural@intrusions
    !pip install lavavu-osmesa geopandas meshio rasterio owslib
    !pip install pyrsistent

else:
    print('Not running on CoLab, nothing to do')

# Tasmanian Dolerites in the Hobart District

### Model 1 - Sill steps only occurred in mapped pre-intrusion faults.

In [ ]:
from datetime import datetime
datetime.now().isoformat(timespec='seconds') 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Loop library
from LoopStructural import GeologicalModel
from LoopStructural.visualisation import lavavu 

In [ ]:
lower_extent = [511530, 5238858, -2500]
upper_extent = [541353, 5267079, 2000]

In [ ]:
if 'google.colab' in str(get_ipython()):
    model_data = pd.read_csv('./loopstructural_intrusions_tassiedolerites/Model1_Data.csv')
    topo = pd.read_csv('./loopstructural_intrusions_tassiedolerites/DEM_Data.csv')
    dem_df = pd.read_csv('./loopstructural_intrusions_tassiedolerites/DEM_Data.csv')
else:
    model_data = pd.read_csv('Model1_Data.csv')
    topo = pd.read_csv('DEM_Data.csv')
    dem_df = pd.read_csv('DEM_Data.csv')

In [ ]:
def elevation_from_z_inteprolation(xy):
    from scipy.interpolate import NearestNDInterpolator
    dtm = dem_df
    x = dtm.loc[:,'X'].to_numpy()
    y = dtm.loc[:,'Y'].to_numpy()
    z = dtm.loc[:,'Z'].to_numpy()
    dtm_bounds = [np.min(x),np.min(y),np.max(x),np.max(y)]


    dtm_interpolator = NearestNDInterpolator((x,y),z)
    interpolated_dtm =np.zeros(xy.shape[0])
    interpolated_dtm[:] = np.nan
    inside = np.logical_and(xy[:,0] > dtm_bounds[0], xy[:,0] < dtm_bounds[2])
    inside = np.logical_and(inside, xy[:,1] > dtm_bounds[1])
    inside = np.logical_and(inside, xy[:,1] < dtm_bounds[3])
    interpolated_dtm = dtm_interpolator(xy[:])
    
    return interpolated_dtm

In [ ]:
topo['val'] = 0
topo['feature_name'] = 'topo'
topo.reset_index(inplace=True)
topo.loc[0,['gx','gy','gz']] = [0,0,1]

model_data = pd.concat([model_data,topo])

In [ ]:
# Create Geological Model

model = GeologicalModel(lower_extent,upper_extent)
model.nsteps = [30,30,30]
model.data = model_data

In [ ]:
topography = model.create_and_add_foliation('topo')

In [ ]:
post_intrusion_fault_1 = model.create_and_add_fault('fault_1',
                                                    displacement=200, 
                                                    #buffer=0.3
                                                   )


post_intrusion_fault_2 = model.create_and_add_fault('fault_2',
                                                    displacement=100, 
                                                    #buffer=0.3
                                                   )

post_intrusion_fault_3a = model.create_and_add_fault('fault_3a',
                                                    displacement=900,  
                                                     #buffer=0.3
                                                    )

post_intrusion_fault_3b = model.create_and_add_fault('fault_3b',
                                                    displacement=400, 
                                                     #buffer=0.3
                                                    )

post_intrusion_fault_3c = model.create_and_add_fault('fault_3c',
                                                    displacement=400,  
                                                     #buffer=0.3
                                                    )

post_intrusion_fault_3d = model.create_and_add_fault('fault_3d',
                                                    displacement=250,
                                                     #buffer=0.3
                                                    )

post_intrusion_fault_3e = model.create_and_add_fault('fault_3e',
                                                    displacement=250, 
                                                     #buffer=0.3
                                                    )
post_intrusion_fault_3f = model.create_and_add_fault('fault_3f',
                                                    displacement=240,   
                                                     buffer=0.3
                                                    )

post_intrusion_fault_4a = model.create_and_add_fault('fault_4a',
                                                    displacement=250, 
                                                     #buffer=0.3
                                                    )

post_intrusion_fault_4b = model.create_and_add_fault('fault_4b',
                                                    displacement=250,
                                                     #buffer=0.3
                                                    )


In [ ]:
post_intrusion_fault_3a.add_abutting_fault(post_intrusion_fault_3b)

post_intrusion_fault_3b.add_abutting_fault(post_intrusion_fault_3c)

post_intrusion_fault_3d.add_abutting_fault(post_intrusion_fault_3e)

In [ ]:
post_intrusion_faults = [post_intrusion_fault_1, post_intrusion_fault_2, 
                         post_intrusion_fault_3a, post_intrusion_fault_3b, post_intrusion_fault_3c,
                         post_intrusion_fault_3d, post_intrusion_fault_3e, post_intrusion_fault_3f,
                         post_intrusion_fault_4a, post_intrusion_fault_4b]

In [ ]:
Parmeener_supergroup = model.create_and_add_foliation('Tasmanian Basin',
                                                      nelements = 3000, 
                                                      solver = 'lu', 
                                                      interpolatortype = 'FDI',
                                                      buffer = 0.8)

# stratigraphic column
stratigraphic_column = {}
stratigraphic_column['Tasmanian Basin'] = {}
stratigraphic_column['Tasmanian Basin']['Pre Lower-Permian units 1'] = {'min':-np.inf,'max':-200,'id':0}
stratigraphic_column['Tasmanian Basin']['Pre Lower-Permian units 2'] = {'min':-200,'max':-100,'id':10}
stratigraphic_column['Tasmanian Basin']['Lower-Permian units'] = {'min':-100,'max':0,'id':1}  # Pln, Plo
stratigraphic_column['Tasmanian Basin']['Mid-Permian units'] = {'min':0,'max':130,'id':2}  # Pff, puc, 
stratigraphic_column['Tasmanian Basin']['Upper-Permian units'] = {'min':130,'max':470,'id':3} # Pum/Pur and Pua
stratigraphic_column['Tasmanian Basin']['Triassic units 1'] = {'min':470,'max':500,'id':4} # Pch, Rqp, Rqm
stratigraphic_column['Tasmanian Basin']['Triassic units 2'] = {'min':500,'max':550,'id':5} # Pch, Rqp, Rqm
stratigraphic_column['Tasmanian Basin']['Triassic units 3'] = {'min':550,'max':600,'id':6} # Pch, Rqp, Rqm
stratigraphic_column['Tasmanian Basin']['Triassic units 4'] = {'min':600,'max':650,'id':7} # Upper triassic
stratigraphic_column['Tasmanian Basin']['Triassic units 5'] = {'min':650,'max':800,'id':8} # Upper triassic
stratigraphic_column['Tasmanian Basin']['Triassic units 6'] = {'min':800,'max':np.inf,'id':9} # Upper triassic

model.set_stratigraphic_column(stratigraphic_column)

In [ ]:
# create pre-intrusion faults
displacement = 0

pre_intrusion_fault_1 = model.create_and_add_fault('pi_fault_1', 
                                                   displacement=displacement
                                                  )

pre_intrusion_fault_2 = model.create_and_add_fault('pi_fault_2', 
                                                   displacement=displacement
                                                  )

In [ ]:
sill1_data = model.data[(model.data['feature_name']=='Sill_1')].loc[:,['X','Y','Z']].to_numpy()
sill2_data = model.data[(model.data['feature_name']=='Sill_2')].loc[:,['X','Y','Z']].to_numpy()
sill3_data = model.data[(model.data['feature_name']=='Sill_3')].loc[:,['X','Y','Z']].to_numpy()

### Visualisation of pre-intrusion units and fault

In [ ]:
datetime.now().isoformat(timespec='seconds')   

In [ ]:
# Visualisation of pre-intrusion units and faults
# viewer1 = lavavu.LavaVuModelViewer(model, background='white')
# nn= 30
# viewer1.nsteps = [nn,nn,nn]

# model.dtm = elevation_from_z_inteprolation

# viewer1.add_isosurface(pre_intrusion_fault_1, isovalue = 0, colour = 'darkred')
# viewer1.add_isosurface(pre_intrusion_fault_2, isovalue = 0, colour = 'darkred')

# viewer1.add_isosurface(topography, isovalue = 0, colour = 'black',  name = 'topo') 

# viewer1.add_isosurface(Parmeener_supergroup, isovalue = -500, colour = 'darkblue',  name = 'Plo1') 
# viewer1.add_isosurface(Parmeener_supergroup, isovalue = -100, colour = 'darkblue',  name = 'Plo2')
# viewer1.add_isosurface(Parmeener_supergroup, isovalue = 0, colour = 'darkblue',  name = 'Base of Pln') 
# viewer1.add_isosurface(Parmeener_supergroup, isovalue = 90, colour = 'royalblue',  name = 'Base of Pff') 
# viewer1.add_isosurface(Parmeener_supergroup, isovalue = 130, colour = 'royalblue', name = 'Pff') 
# viewer1.add_isosurface(Parmeener_supergroup, isovalue = 240, colour = 'turquoise',  name = 'Base of Pum') 
# viewer1.add_isosurface(Parmeener_supergroup, isovalue = 310, colour = 'turquoise',  name = 'Base of Pua') 
# viewer1.add_isosurface(Parmeener_supergroup, isovalue = 470, colour = 'green',  name = 'Base of Triassic') 
# viewer1.add_isosurface(Parmeener_supergroup, isovalue = 500, colour = 'green',  name = 'Triassic 500') 
# viewer1.add_isosurface(Parmeener_supergroup, isovalue = 520, colour = 'green',  name = 'Triassic 520') 
# viewer1.add_isosurface(Parmeener_supergroup, isovalue = 600, colour = 'green',  name = 'Triassic 600')
# viewer1.add_isosurface(Parmeener_supergroup, isovalue = 620, colour = 'green',  name = 'Triassic 620') 
# viewer1.add_isosurface(Parmeener_supergroup, isovalue = 650, colour = 'green',  name = 'Triassic 650') 
# viewer1.add_isosurface(Parmeener_supergroup, isovalue = 700, colour = 'green',  name = 'Triassic 700') 
# # viewer1.add_data(Parmeener_supergroup)

# # viewer1.add_isosurface(Parmeener_supergroup, nslices = 20, paint_with = Parmaneer_supergroup) 

# viewer1.add_isosurface(post_intrusion_fault_1, isovalue = 0, colour = 'black')
# viewer1.add_isosurface(post_intrusion_fault_2, isovalue = 0, colour = 'black')
# viewer1.add_isosurface(post_intrusion_fault_3a, isovalue = 0, colour = 'black')
# viewer1.add_isosurface(post_intrusion_fault_3b, isovalue = 0, colour = 'black')
# viewer1.add_isosurface(post_intrusion_fault_3c, isovalue = 0, colour = 'black')
# viewer1.add_isosurface(post_intrusion_fault_3d, isovalue = 0, colour = 'black')
# viewer1.add_isosurface(post_intrusion_fault_3e, isovalue = 0, colour = 'black')
# viewer1.add_isosurface(post_intrusion_fault_3f, isovalue = 0, colour = 'black')
# viewer1.add_isosurface(post_intrusion_fault_4a, isovalue = 0, colour = 'black')
# viewer1.add_isosurface(post_intrusion_fault_4b, isovalue = 0, colour = 'black')

# viewer1.ymin=0
# viewer1.ymax=1
# viewer1.xmin=0
# viewer1.xmax=1
# viewer1.rotate([-60,-30,0])
# # viewer1.rotate([-42.978, -28.767, -14.57])
# viewer1.interactive()

In [ ]:
datetime.now().isoformat(timespec='seconds')   

### Conceptual Models

In [ ]:
# conceptual geometrical models

def ellipse_function(
    lateral_contact_data=pd.DataFrame(), 
    model = True, # True to cover the extent of the model, regardless of data distribution
    minP=None, 
    maxP=None, 
    minS=None, 
    maxS=None):
    
    if lateral_contact_data.empty:
        return model, minP, maxP, minS, maxS

    else:
        
        if minP == None:
            minP = lateral_contact_data["coord1"].min()
        if maxP == None:
            maxP = lateral_contact_data["coord1"].max()
        if minS == None:
            minS = lateral_contact_data["coord2"].abs().min()
        if maxS == None:
            maxS = lateral_contact_data["coord2"].max()

        a = (maxP - minP) / 2
        b = (maxS - minS) / 2

        po = minP + (maxP - minP) / 2

        p_locations = lateral_contact_data.loc[:, "coord1"].copy().to_numpy()

        s = np.zeros([len(p_locations), 2])

        s[np.logical_and(p_locations > minP, p_locations < maxP), 0] = b * np.sqrt(
            1
            - np.power(
                (p_locations[np.logical_and(p_locations > minP, p_locations < maxP)] - po)
                / a,
                2,
            )
        )
        s[np.logical_and(p_locations > minP, p_locations < maxP), 1] = -b * np.sqrt(
            1
            - np.power(
                (p_locations[np.logical_and(p_locations > minP, p_locations < maxP)] - po)
                / a,
                2,
            )
        )

        return s
    
def constant_function(
    othercontact_data = pd.DataFrame(),
    mean_growth=None, 
    minP=None, 
    maxP=None, 
    minS=None, 
    maxS=None, 
    vertex=None):
    
    if othercontact_data.empty:
        return mean_growth, vertex
    
    if mean_growth == None:
        mean_growth = othercontact_data.loc[:,'coord1'].mean()
        
    data_ps = np.array([othercontact_data.loc[:,'coord1'], othercontact_data.loc[:,'coord2']]).T
    
    conceptual_growth = np.ones([len(data_ps),2]) * mean_growth
    
    return conceptual_growth


In [ ]:
# create sill 1 - floor in Triassic

intrusion_frame_parameters = {'contact' :'floor',
                              'contact_anisotropies' : [Parmeener_supergroup],
                              'g_w':0.05
                             }

Sill_Segment_1 = model.create_and_add_intrusion('Sill_1', intrusion_frame_name = 'Sill_1_frame',
                                                intrusion_lateral_extent_model = ellipse_function,
                                                intrusion_vertical_extent_model = constant_function,
                                                intrusion_frame_parameters = intrusion_frame_parameters,
                                                faults = post_intrusion_faults,
                                                geometric_scaling_parameters = {'intrusion_type' : 'major_mafic_sills', 
                                                                                'intrusion_length' : 5e4
                                                                               },
                                                interpolatortype = 'FDI',
                                                buffer = 1
                                               )

In [ ]:
# create sill 2 - floor in Upper Permian
intrusion_steps = {}

intrusion_steps['step1'] = {'structure' : pre_intrusion_fault_1, 
                            'unit_from' : 'Mid-Permian units', 'series_from': Parmeener_supergroup, 
                            'unit_to' : 'Triassic units 2','series_to': Parmeener_supergroup}


intrusion_frame_parameters = {'contact' :'floor',
                              'intrusion_steps' : intrusion_steps,
                              'contact_anisotropies' : [Parmeener_supergroup],
                              'delta_c' : [[1,1]],
                              'delta_f' : [5],
                              'g_w':0.01
                               }

Sill_Segment_2 = model.create_and_add_intrusion('Sill_2', intrusion_frame_name = 'Sill_2_frame',
                                                intrusion_lateral_extent_model = ellipse_function,
                                                intrusion_vertical_extent_model = constant_function,
                                                intrusion_frame_parameters = intrusion_frame_parameters,
                                                faults = post_intrusion_faults,
                                                geometric_scaling_parameters = {'intrusion_type' : 'major_mafic_sills', 
                                                                                'intrusion_length' : 7e4
                                                                               },
                                                interpolatortype = 'FDI',
                                                buffer = 1
                                               )

In [ ]:
# create sill 3 - roof in mid Permian
intrusion_steps = {}
intrusion_steps['step1'] = {'structure' : pre_intrusion_fault_2, 
                            'unit_from' : 'Mid-Permian units', 'series_from': Parmeener_supergroup, 
                            'unit_to' : 'Triassic units 2','series_to': Parmeener_supergroup}

intrusion_frame_parameters = {'contact' :'roof',
                              'intrusion_steps' : intrusion_steps,
                              'contact_anisotropies' : [Parmeener_supergroup],
                              'delta_c' : [[1,1]],
                              'delta_f' : [5],
                              'g_w':0.01
                               }

Sill_Segment_3 = model.create_and_add_intrusion('Sill_3', intrusion_frame_name = 'Sill_3_frame',
                                                intrusion_lateral_extent_model = ellipse_function,
                                                intrusion_vertical_extent_model = constant_function,
                                                intrusion_frame_parameters = intrusion_frame_parameters,
                                                geometric_scaling_parameters = {'intrusion_type' : 'major_mafic_sills',
                                                                                'intrusion_length' : 7e4
                                                                               },
                                                faults = post_intrusion_faults,
                                                interpolatortype = 'FDI',
                                                buffer = 1
                                               )

In [ ]:
datetime.now().isoformat(timespec='seconds')   

In [ ]:
def outside_sills(xyz):
    sill1_val = Sill_Segment_1.evaluate_value(xyz)
    sill1_inside = sill1_val>=0
    sill2_val = Sill_Segment_2.evaluate_value(xyz)
    sill2_inside = sill2_val>=0
    sill3_val = Sill_Segment_3.evaluate_value(xyz)
    sill3_inside = sill3_val>=0
    
    inside_any_sill = np.logical_or(sill1_inside, np.logical_or(sill2_inside,sill3_inside))
    inside_any_sill = inside_any_sill.astype(int) #1=inside ; 0 =outside
    
    v = np.zeros_like(inside_any_sill, dtype = bool)
    
    v[inside_any_sill==0] = True
    v[inside_any_sill==1] = False
    return v


def below_topo(xyz):
    topo_vals = topography.evaluate_value(xyz)
    
    v = np.zeros_like(topo_vals, dtype = bool)
    
    v[topo_vals<=0] = True
    v[topo_vals>0] = False
    
    return v
    


In [ ]:
datetime.now().isoformat(timespec='seconds')

In [ ]:
# viewer = lavavu.LavaVuModelViewer(model, background='white')
# nn = 10 #50
# viewer.nsteps = [nn,nn,nn*2]

# # -- add stratigraphy

# viewer.add_isosurface(topography, isovalue = 0, paint_with = Sill_Segment_1)

# # add intrusion
# viewer.add_isosurface(Sill_Segment_1, isovalue = 0, colour = 'orange') 
# viewer.add_isosurface(Sill_Segment_2, isovalue = 0, colour = 'goldenrod')  
# viewer.add_isosurface(Sill_Segment_3, isovalue = 0, colour = 'darkorange') 

# # viewer.add_points(model.rescale(sill1_data, inplace = False), name = 'sill1_data', pointsize = 4)
# # viewer.add_points(model.rescale(sill2_data, inplace = False), name = 'sill2_data', pointsize = 4)
# # viewer.add_points(model.rescale(sill3_data, inplace = False), name = 'sill3_data', pointsize = 4)

# viewer.ymin = 0
# viewer.ymax = 1
# viewer.xmin = 0
# viewer.xmax = 1
# viewer.rotate([-60, -30, 0])
# viewer.interactive()

In [ ]:
# visualization of 3D model of dolerites post post-intrusion faulting


viewer = lavavu.LavaVuModelViewer(model, background='white')
nn = 50
viewer.nsteps = [nn,nn,nn*2]

# -- add stratigraphy

# viewer.add_isosurface(topography, isovalue = 0, colour='black')
model.dtm = elevation_from_z_inteprolation
viewer.add_isosurface(Parmeener_supergroup, isovalue = -500, colour = 'darkblue',  name = 'Plo1', region = outside_sills) 
viewer.add_isosurface(Parmeener_supergroup, isovalue = -100, colour = 'darkblue',  name = 'Plo2', region = outside_sills)
viewer.add_isosurface(Parmeener_supergroup, isovalue = 0, colour = 'darkblue',  name = 'Base of Pln', region = outside_sills) 
viewer.add_isosurface(Parmeener_supergroup, isovalue = 90, colour = 'royalblue',  name = 'Base of Pff', region = outside_sills) 
viewer.add_isosurface(Parmeener_supergroup, isovalue = 130, colour = 'royalblue', name = 'Pff', region = outside_sills) 
viewer.add_isosurface(Parmeener_supergroup, isovalue = 240, colour = 'turquoise',  name = 'Base of Pum', region = outside_sills) 
viewer.add_isosurface(Parmeener_supergroup, isovalue = 310, colour = 'turquoise',  name = 'Base of Pua', region = outside_sills) 
viewer.add_isosurface(Parmeener_supergroup, isovalue = 470, colour = 'green',  name = 'Base of Triassic', region = outside_sills) 
viewer.add_isosurface(Parmeener_supergroup, isovalue = 600, colour = 'green',  name = 'Triassic 1', region = outside_sills) 
viewer.add_isosurface(Parmeener_supergroup, isovalue = 800, colour = 'green',  name = 'Triassic 2', region = outside_sills) 


# -- add pre-intrusion faults
viewer.add_isosurface(pre_intrusion_fault_1, isovalue = 0, colour = 'red')
viewer.add_isosurface(pre_intrusion_fault_2, isovalue = 0, colour = 'red')

# # -- add post-intrusion faults
viewer.add_isosurface(post_intrusion_fault_1, isovalue = 0, colour = 'black')
viewer.add_isosurface(post_intrusion_fault_2, isovalue = 0, colour = 'black')
viewer.add_isosurface(post_intrusion_fault_3a, isovalue = 0, colour = 'black')
viewer.add_isosurface(post_intrusion_fault_3b, isovalue = 0, colour = 'black')
viewer.add_isosurface(post_intrusion_fault_3c, isovalue = 0, colour = 'black')
viewer.add_isosurface(post_intrusion_fault_3d, isovalue = 0, colour = 'black')
viewer.add_isosurface(post_intrusion_fault_3e, isovalue = 0, colour = 'black')
viewer.add_isosurface(post_intrusion_fault_3f, isovalue = 0, colour = 'black')
viewer.add_isosurface(post_intrusion_fault_4a, isovalue = 0, colour = 'black')
viewer.add_isosurface(post_intrusion_fault_4b, isovalue = 0, colour = 'black')


# add intrusion
viewer.add_isosurface(Sill_Segment_1, isovalue = 0, colour = 'orange') 
viewer.add_isosurface(Sill_Segment_2, isovalue = 0, colour = 'goldenrod')  
viewer.add_isosurface(Sill_Segment_3, isovalue = 0, colour = 'darkorange') 

viewer.add_points(model.rescale(sill1_data, inplace = False), name = 'sill1_data', pointsize = 4)
viewer.add_points(model.rescale(sill2_data, inplace = False), name = 'sill2_data', pointsize = 4)
viewer.add_points(model.rescale(sill3_data, inplace = False), name = 'sill3_data', pointsize = 4)

viewer.ymin = 0
viewer.ymax = 1
viewer.xmin = 0
viewer.xmax = 1
viewer.rotate([-60, -30, 0])
viewer.interactive()

In [ ]:
datetime.now().isoformat(timespec='seconds')   